I want a script that takes an amino acid sequence as input (FASTA format), and creates a codon optimized DNA sequence as output (FASTA). 

It should be optimized based on an excel file specifying the codon usage, the user should be able to specify forbidden sequences (e.g. restriction sites) that the script avoids, and it should keep the sequence within an acceptable GC content (30-80%), both locally (specified window size, e.g. 15 bp) and globally (whole sequence). 

In the end the script should double check that the DNA generated translates to the AA sequence provided as input. 

You will need to use the Pandas package for reading the excel/working with the dataframe.

and you will need Biopython for reading FASTA format and doing some DNA-related operations (translate, gc-content, etc.)

To break it down in to smaller parts, I would recommend to simply start with reading the FASTA and Excel input. 

as a next step you can try building the DNA sequence (maybe even without considering codon preference) and verifying that it translates correctly. 

Then you can continue by considering codon preference.

and lastly avoiding the forbidden sequences


1. Read FASTA and Excel Input:

Read the amino acid sequence from the FASTA file.
Read the codon usage information from the Excel file. You can use libraries like pandas to handle Excel files.

2. Generate DNA Sequence without Codon Optimization:

Create a DNA sequence based on the input amino acid sequence without considering codon preference. This can be done using a simple translation table from amino acids to codons.

3. Verify Translation:

Translate the generated DNA sequence back to an amino acid sequence.
Compare the translated amino acid sequence with the input amino acid sequence to ensure they match.

4. Codon Optimization:

Implement a codon optimization algorithm based on the codon usage information from the Excel file. You can use optimization techniques such as Genetic Algorithms or other heuristics to find the best codon for each amino acid while avoiding forbidden sequences.

5. GC Content Control:

Calculate the GC content of the generated DNA sequence in a sliding window of the specified size.
If the GC content falls outside the acceptable range (20-80%) for any part of the sequence, make necessary adjustments.

6. Avoid Forbidden Sequences:

Check the generated DNA sequence for the presence of forbidden sequences (e.g., restriction sites) and make modifications to avoid them. You may need to implement a search and replace mechanism for this.

7. Iterate and Optimize:

Repeat steps 4-6 until you have a DNA sequence that meets all the criteria.

8. Write Output to FASTA:

Write the optimized DNA sequence to a FASTA file.

# Codon as input

### 1. Codon table

In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import random

In [2]:
   # Read the Excel file into a DataFrame
codon_table = pd.read_excel(r'C:\Users\Candela\OneDrive - Danmarks Tekniske Universitet\Codopti\codon_table.xlsx', sheet_name=0)

Visualise the codon table.

In [3]:
print(codon_table.tail()) 

   Amino Acid Codon  Frequency
59          V   GTG        8.5
60          V   GTT       33.9
61          W   TGG      100.0
62          Y   TAC       95.1
63          Y   TAT        4.9


Find codons with <5% frequency in Yarrowia and remove them.

In [4]:
def cod_opt_table(codon_table, output_excel_path):

    # Filter out codons with a frequency less than 5.0
    codon_table = codon_table[codon_table['Frequency'] >= 5.0]

    # Write the updated DataFrame to a new Excel file
    codon_table.to_excel(output_excel_path)

# Example usage:
opt_codon_table = r'C:\Users\Candela\OneDrive - Danmarks Tekniske Universitet\Codopti\opt_codon_table.xlsx'  # Replace with the desired output path

cod_opt_table(codon_table, opt_codon_table)

Recalculate the frequency of the codons for each amino acid.

In [5]:
def calculate_new_frequencies(codon_table):
    # Group the DataFrame by Amino Acid
    grouped = codon_table.groupby('Amino Acid')

    # Calculate the total frequency for each amino acid
    amino_acid_totals = grouped['Frequency'].transform('sum')

    # Calculate the new frequencies for each codon based on the rule of three
    codon_table['New Frequency'] = (
        codon_table['Frequency'] / amino_acid_totals * 100
    )

    return codon_table

Calculate and add the lower and upper bounds to the excel table:

In [6]:
# Calculate the new frequencies
updated_opt_codon_table = calculate_new_frequencies(pd.read_excel(opt_codon_table))

# Print the updated DataFrame (with new frequencies)
print(updated_opt_codon_table)

    Unnamed: 0 Amino Acid Codon  Frequency  New Frequency
0            0          -   TAA       83.0      83.000000
1            1          -   TAG       10.0      10.000000
2            2          -   TGA        7.0       7.000000
3            4          A   GCC       59.1      60.927835
4            6          A   GCT       37.9      39.072165
5            7          C   TGC       65.6      65.600000
6            8          C   TGT       34.4      34.400000
7            9          D   GAC       70.0      70.000000
8           10          D   GAT       30.0      30.000000
9           11          E   GAA        6.4       6.400000
10          12          E   GAG       93.6      93.600000
11          13          F   TTC       80.7      80.700000
12          14          F   TTT       19.3      19.300000
13          15          G   GGA       16.2      16.413374
14          16          G   GGC       30.6      31.003040
15          18          G   GGT       51.9      52.583587
16          19

### 2. Sequence


Downloaded a random fasta sequence that we will codon optimise.

In [53]:
my_fasta = r'C:\Users\Candela\OneDrive - Danmarks Tekniske Universitet\Codopti\YALI2.txt' 

Visualise your amino acid fasta sequence:

In [8]:
for seq_record in SeqIO.parse(my_fasta, "fasta"):
    print(seq_record.id)
    print(len(seq_record))
    print(repr(seq_record))
    name = seq_record.id
    AA_seq = seq_record.seq

YALI0_D05621g
592
SeqRecord(seq=Seq('MAHDSELELSDEKVVPSINQEKHSFFQRHLDNHPRMAQYNSQLQRFLKWIEVPT...IIS'), id='YALI0_D05621g', name='YALI0_D05621g', description='YALI0_D05621g  | Yarrowia lipolytica CLIB122 | YALI0D05621p | protein | length=592', dbxrefs=[])


Use the updated_opt_codon_table to reverse translate the amino acid sequence:

In [52]:
print(updated_opt_codon_table)

    Unnamed: 0 Amino Acid Codon  Frequency  New Frequency
0            0          -   TAA       83.0      83.000000
1            1          -   TAG       10.0      10.000000
2            2          -   TGA        7.0       7.000000
3            4          A   GCC       59.1      60.927835
4            6          A   GCT       37.9      39.072165
5            7          C   TGC       65.6      65.600000
6            8          C   TGT       34.4      34.400000
7            9          D   GAC       70.0      70.000000
8           10          D   GAT       30.0      30.000000
9           11          E   GAA        6.4       6.400000
10          12          E   GAG       93.6      93.600000
11          13          F   TTC       80.7      80.700000
12          14          F   TTT       19.3      19.300000
13          15          G   GGA       16.2      16.413374
14          16          G   GGC       30.6      31.003040
15          18          G   GGT       51.9      52.583587
16          19

In [48]:

def reverse_translate_amino_acids_fasta(AA_seq, codon_table):
    # Initialize an empty DNA sequence
    dna_sequence = Seq("")
    
    # Remove the header line from the FASTA format
    amino_acid_sequence = AA_seq
    
    print(amino_acid_sequence)

    for amino_acid in amino_acid_sequence:
        # Filter the codon table for the current amino acid
        amino_acid_codons = codon_table[codon_table['Amino Acid'] == amino_acid]

        if not amino_acid_codons.empty:
            # Assign codon value ranges based on their frequencies
            value_ranges = []
            lower_bound = 0
            for _, row in amino_acid_codons.iterrows():
                upper_bound = lower_bound + row['New Frequency']
                value_ranges.append((row['Codon'], lower_bound, upper_bound))
                lower_bound = upper_bound

            # Generate a random value within the specified range
            random_value = random.uniform(0, 100)

            # Map the random value to the corresponding codon
            for codon, lower, upper in value_ranges:
                if lower <= random_value <= upper:
                    dna_sequence += Seq(codon)
                    break

    return dna_sequence

# Call the function to reverse translate the amino acid sequence
reverse_translated_dna = reverse_translate_amino_acids_fasta(AA_seq, updated_opt_codon_table)

# Create a SeqRecord object for the DNA sequence
dna_record = SeqRecord(reverse_translated_dna, id="reverse_translated_sequence")

# Print the SeqRecord or save it to a FASTA file
print(dna_record.format("fasta"))


MAHDSELELSDEKVVPSINQEKHSFFQRHLDNHPRMAQYNSQLQRFLKWIEVPTKEGEINTFLNNEDLKPVEVARQTWGWKNFVSFWIADSFNINTWEIAATGIQLGLTWWQVWLCVWIGYFFCGVFVVLSGRIGAIYHVSFPVAGRSTFGIFGSIWPVINRVVMACVWYGVQGWLGGQCIQVCLLAIWPSARHMKNGIPGSGTTTFEFLSYFLFWLFSLPFIYIRPHNLRHLFMVKAAIVPVAGISFLVWTCVKAHGIGPIMKQPATVHGSVMGWAFMTAIMNSLSNFATIIVNAPDFTRFAKEPNAIVLSQLIAVPTAFSLTSFIGIIVSSSATVLYDENIWNPLDVLHKFLEGNKSGSRAGVFFLGFAFAVAQLGTNIAANSLSAGTDMTALLPKYINIRRGGFICAGIALCICPWHLLSSSSNFTTYLSAYATFLSAIAGCSFSDYYLVRKGYIYVGDLYNASKGSTYMYRYGVNWRAFAAYFCGIAINVVGFADAVSDGGVNETARKMYQLNFFLGFLVSAISYYGFNWLSPVVGARETWSEDPNASAMYDEITTDELSQDSQSYDPEEWDRKIANDDPVKTTAIIS
>reverse_translated_sequence <unknown description>
ATGGCTCATGACTCTGAGCTGGAGCTCTCTGACGAGAAGGTCGTCCCCTCTATCAACCAG
GAGAAGCACTCTTTTTTTCAGCGACACCTGGACAACCACCCCCGAATGGCCCAGTACAAC
TCCCAGCTCCAGCGATTCCTGAAGTGGATCGAGGTCCCCACCAAGGAGGGCGAGATCAAC
ACTTTTCTCAACAACGAGGACCTGAAGCCCGTCGAGGTCGCTCGACAGACTTGGGGCTGG
AAGAACTTCGTCTCCTTCTGGATCGCCGACTCCTTCAACATTAACACCTGGGAAATCGCT
GCCACCGGCATTCAGCTCGGTCTCACCTGGTGGCAGGTCTGGCTCTGCGTT

In [10]:
def reverse_translate_amino_acids_fasta(AA_seq, codon_table):
    # Define the forbidden sequences
    forbidden_sequences = ["GCTCTTCN", "GCGGCCGC", "GGTCTCNNNNN"]
    
    # Initialize an empty DNA sequence
    dna_sequence = Seq("")
    
    # Remove the header line from the FASTA format
    amino_acid_sequence = AA_seq
    
    print(amino_acid_sequence)

    while True:
        # Initialize a list to keep track of added codons
        added_codons = []
        
        for amino_acid in amino_acid_sequence:
            # Filter the codon table for the current amino acid
            amino_acid_codons = codon_table[codon_table['Amino Acid'] == amino_acid]

            if not amino_acid_codons.empty:
                # Assign codon value ranges based on their frequencies
                value_ranges = []
                lower_bound = 0
                for _, row in amino_acid_codons.iterrows():
                    upper_bound = lower_bound + row['New Frequency']
                    value_ranges.append((row['Codon'], lower_bound, upper_bound))
                    lower_bound = upper_bound

                while True:
                    # Generate a random value within the specified range
                    random_value = random.uniform(0, 100)

                    # Map the random value to the corresponding codon
                    selected_codon = None
                    for codon, lower, upper in value_ranges:
                        if lower <= random_value <= upper:
                            selected_codon = codon
                            break
                    
                    # Check if the selected codon contains any forbidden sequences
                    contains_forbidden_sequence = any(seq in selected_codon for seq in forbidden_sequences)
                    
                    if not contains_forbidden_sequence:
                        # Add the selected codon to the DNA sequence
                        dna_sequence += Seq(selected_codon)
                        added_codons.append(selected_codon)
                        break
        
        # Check if the generated sequence contains any forbidden sequences
        contains_forbidden_sequence = any(seq in dna_sequence for seq in forbidden_sequences)
        
        if not contains_forbidden_sequence:
            break  # Exit the loop if the sequence is valid
    
    return dna_sequence

# Call the function to reverse translate the amino acid sequence
reverse_translated_dna = reverse_translate_amino_acids_fasta(AA_seq, updated_opt_codon_table)

# Create a SeqRecord object for the DNA sequence
dna_record = SeqRecord(reverse_translated_dna, id="reverse_translated_sequence")

# Print the SeqRecord or save it to a FASTA file
print(dna_record.format("fasta"))

MAHDSELELSDEKVVPSINQEKHSFFQRHLDNHPRMAQYNSQLQRFLKWIEVPTKEGEINTFLNNEDLKPVEVARQTWGWKNFVSFWIADSFNINTWEIAATGIQLGLTWWQVWLCVWIGYFFCGVFVVLSGRIGAIYHVSFPVAGRSTFGIFGSIWPVINRVVMACVWYGVQGWLGGQCIQVCLLAIWPSARHMKNGIPGSGTTTFEFLSYFLFWLFSLPFIYIRPHNLRHLFMVKAAIVPVAGISFLVWTCVKAHGIGPIMKQPATVHGSVMGWAFMTAIMNSLSNFATIIVNAPDFTRFAKEPNAIVLSQLIAVPTAFSLTSFIGIIVSSSATVLYDENIWNPLDVLHKFLEGNKSGSRAGVFFLGFAFAVAQLGTNIAANSLSAGTDMTALLPKYINIRRGGFICAGIALCICPWHLLSSSSNFTTYLSAYATFLSAIAGCSFSDYYLVRKGYIYVGDLYNASKGSTYMYRYGVNWRAFAAYFCGIAINVVGFADAVSDGGVNETARKMYQLNFFLGFLVSAISYYGFNWLSPVVGARETWSEDPNASAMYDEITTDELSQDSQSYDPEEWDRKIANDDPVKTTAIIS
>reverse_translated_sequence <unknown description>
ATGGCCCATGACTCTGAGCTCGAACTCTCCGATGAGAAGGTTGTCCCCTCCATCAACCAG
GAGAAGCACTCTTTTTTCCAGCGACACCTCGACAACCATCCCCGAATGGCCCAGTACAAC
TCCCAGCTCCAGCGATTCCTGAAGTGGATCGAGGTCCCTACCAAGGAGGGAGAGATCAAC
ACCTTCCTTAACAACGAGGACCTCAAGCCCGTGGAGGTCGCTCGACAGACCTGGGGTTGG
AAGAACTTCGTCTCTTTCTGGATCGCCGACTCCTTTAACATCAACACCTGGGAGATCGCC
GCCACTGGTATCCAGCTCGGTCTCACCTGGTGGCAGGTTTGGCTCTGCGTC

Check that the sequence changes every time you run it:

In [11]:
print(dna_record.seq)

ATGGCCCATGACTCTGAGCTCGAACTCTCCGATGAGAAGGTTGTCCCCTCCATCAACCAGGAGAAGCACTCTTTTTTCCAGCGACACCTCGACAACCATCCCCGAATGGCCCAGTACAACTCCCAGCTCCAGCGATTCCTGAAGTGGATCGAGGTCCCTACCAAGGAGGGAGAGATCAACACCTTCCTTAACAACGAGGACCTCAAGCCCGTGGAGGTCGCTCGACAGACCTGGGGTTGGAAGAACTTCGTCTCTTTCTGGATCGCCGACTCCTTTAACATCAACACCTGGGAGATCGCCGCCACTGGTATCCAGCTCGGTCTCACCTGGTGGCAGGTTTGGCTCTGCGTCTGGATCGGTTACTTCTTCTGCGGAGTCTTCGTCGTTCTGTCTGGTCGAATTGGTGCCATCTACCACGTCTCCTTTCCCGTTGCCGGCCGATCTACCTTTGGTATCTTCGGCTCCATCTGGCCCGTCATTAACCGAGTCGTGATGGCCTGCGTGTGGTACGGAGTCCAGGGTTGGCTTGGCGGTCAGTGCATCCAGGTCTGCCTCCTGGCTATTTGGCCTTCTGCCCGACACATGAAGAACGGTATTCCCGGTTCTGGTACCACTACCTTTGAGTTCCTGTCCTACTTCCTGTTCTGGCTCTTCTCCCTTCCCTTTATCTACATTCGACCCCACAACCTCCGACACCTTTTCATGGTTAAGGCCGCCATCGTCCCCGTTGCCGGTATCTCTTTCCTCGTGTGGACTTGTGTTAAGGCCCACGGAATTGGCCCCATTATGAAGCAGCCCGCCACCGTCCACGGCTCCGTCATGGGATGGGCCTTCATGACCGCTATCATGAACTCTCTCTCTAACTTCGCCACCATTATCGTTAACGCTCCTGACTTCACTCGATTCGCCAAGGAGCCCAACGCTATTGTCCTGTCTCAGCTCATCGCTGTCCCCACCGCCTTCTCTCTCACTTCTTTCATCGGTATCATCGTGTCCTCCT

In [12]:
print(dna_record.seq)

ATGGCCCATGACTCTGAGCTCGAACTCTCCGATGAGAAGGTTGTCCCCTCCATCAACCAGGAGAAGCACTCTTTTTTCCAGCGACACCTCGACAACCATCCCCGAATGGCCCAGTACAACTCCCAGCTCCAGCGATTCCTGAAGTGGATCGAGGTCCCTACCAAGGAGGGAGAGATCAACACCTTCCTTAACAACGAGGACCTCAAGCCCGTGGAGGTCGCTCGACAGACCTGGGGTTGGAAGAACTTCGTCTCTTTCTGGATCGCCGACTCCTTTAACATCAACACCTGGGAGATCGCCGCCACTGGTATCCAGCTCGGTCTCACCTGGTGGCAGGTTTGGCTCTGCGTCTGGATCGGTTACTTCTTCTGCGGAGTCTTCGTCGTTCTGTCTGGTCGAATTGGTGCCATCTACCACGTCTCCTTTCCCGTTGCCGGCCGATCTACCTTTGGTATCTTCGGCTCCATCTGGCCCGTCATTAACCGAGTCGTGATGGCCTGCGTGTGGTACGGAGTCCAGGGTTGGCTTGGCGGTCAGTGCATCCAGGTCTGCCTCCTGGCTATTTGGCCTTCTGCCCGACACATGAAGAACGGTATTCCCGGTTCTGGTACCACTACCTTTGAGTTCCTGTCCTACTTCCTGTTCTGGCTCTTCTCCCTTCCCTTTATCTACATTCGACCCCACAACCTCCGACACCTTTTCATGGTTAAGGCCGCCATCGTCCCCGTTGCCGGTATCTCTTTCCTCGTGTGGACTTGTGTTAAGGCCCACGGAATTGGCCCCATTATGAAGCAGCCCGCCACCGTCCACGGCTCCGTCATGGGATGGGCCTTCATGACCGCTATCATGAACTCTCTCTCTAACTTCGCCACCATTATCGTTAACGCTCCTGACTTCACTCGATTCGCCAAGGAGCCCAACGCTATTGTCCTGTCTCAGCTCATCGCTGTCCCCACCGCCTTCTCTCTCACTTCTTTCATCGGTATCATCGTGTCCTCCT

Check for forbidden sequences:


DIct No sapI (7nt), bsaI, notI, no stretches of the same nucleotide max 8, GC content (check 12 nt every new codon added), also check the reverse

In [13]:
from Bio.SeqUtils import GC
GC(dna_record.seq)

55.461711711711715

Translate the sequence back:

In [14]:
dna_record_new = dna_record.translate() 
print(dna_record_new)

ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
/molecule_type=protein
Seq('MAHDSELELSDEKVVPSINQEKHSFFQRHLDNHPRMAQYNSQLQRFLKWIEVPT...IIS')


Compare two sequences: 

In [15]:
dna_record_new.seq == AA_seq

True

--------------------------------

In [16]:
def reverse_translate_amino_acids(amino_acid_sequence, codon_table):
    dna_sequence = Seq("")
    for amino_acid in amino_acid_sequence:
        # Filter the codon table for the current amino acid
        amino_acid_codons = codon_table[codon_table['Amino Acid'] == amino_acid]

        if not amino_acid_codons.empty:
            # Assign codon value ranges based on their frequencies
            value_ranges = []
            lower_bound = 0
            for _, row in amino_acid_codons.iterrows():
                upper_bound = lower_bound + row['New Frequency']
                value_ranges.append((row['Codon'], lower_bound, upper_bound))
                lower_bound = upper_bound

            # Generate a random value within the specified range
            random_value = random.uniform(0, 100)

            # Map the random value to the corresponding codon
            for codon, lower, upper in value_ranges:
                if lower <= random_value <= upper:
                    dna_sequence += Seq(codon)
                    break

    # Ensure that the DNA sequence matches the length of the amino acid sequence
    while len(dna_sequence) < len(amino_acid_sequence) * 3:
        # If it's shorter, add random codons (you can customize this part)
        random_codon = random.choice(codon_table['Codon'])
        dna_sequence += random_codon

    # Trim the sequence to match the desired length
    dna_sequence = dna_sequence[:len(amino_acid_sequence) * 3]

    return dna_sequence

--------------------------------


The table contains three columns:

Amino acid: The amino acid encoded by a given codon (“-“ is for stop codon)

Codon: The codon in question

Frequency: The frequency of a given codon when a specific AA in encoded. 

(If the following is too confusing/complicated then just get started with the other things first, and then get back to this later.)

For example alanine (A), can be encoded by 4 codons (GCA, GCC, GCG, GCT). GCC and GCT occur frequently (59% and 38%), whereas GCA and GCG are very rarely used (2% and 1%). So we can see that Yarrowia has a preference GCC and GCT in regards to alanine.
Biologically speaking this also correlates with the number of tRNA genes Yarrowia has for a given codon.

When doing codon optimization, we don’t want to use the rare codons at all, and we would like to frequent codons to be selected according to their occurrence by using a random number generator.

I would remove the rare codons, and recalculate the frequency for the common codons. You can then proceed in slightly different ways, but you could assign the codons values from 0-100 based on their frequency.

In the case of alanine it might look like this:
After removing the rare codons, the new frequency is:
            GCC = 60.9 %

GCT = 39.1 %

So lets assign GCC the values 0-60.9, and GCT 61.0-100.

We input the sequence AAA, and iterate through it, randomizing a number from 0-100 each time.

We might get the numbers 12, 89, and 35. This would then correspond to the sequence GCC + GCT + GCC.